In [26]:
!pip install yfinance
!pip install pandas_ta
!pip install backtesting
!pip install plotly
!pip install matplotlib
!pip install backtesting

In [2]:
import pandas_ta as ta
import backtesting
backtesting.set_bokeh_output(notebook=False)
from backtesting import Backtest,Strategy
from backtesting.lib import crossover
from bokeh.models import DatetimeTickFormatter
import pandas as pd
import numpy as np
from backtesting.test import GOOG
import yfinance as yf
import plotly.express as px
import matplotlib.pyplot as plt
import datetime

/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [3]:
data = yf.download(tickers="NVDA", start="2024-12-16",end="2025-1-14",interval="30m")
df=pd.DataFrame(data)
df

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA
Datetime,,,,,
2024-12-16 09:30:00,132.519897,134.399994,130.449997,134.164993,49916873
2024-12-16 10:00:00,132.290100,132.859100,130.419998,132.509995,23941819
2024-12-16 10:30:00,131.550003,132.460007,131.330093,132.300003,14103584
2024-12-16 11:00:00,130.929901,132.169998,130.889999,131.550507,13065794
2024-12-16 11:30:00,131.380096,131.429993,130.690094,130.920105,11423976
...,...,...,...,...,...
2025-01-13 13:30:00,132.139999,132.630005,132.009995,132.520004,5925261
2025-01-13 14:00:00,132.294998,132.397003,131.570007,132.130203,7234556


In [4]:
df = df.droplevel(level='Ticker', axis=1)
df.columns.name = None
df = df.reset_index()
dates= df['Datetime'] + pd.Timedelta(hours=5, minutes=30)
df=df.drop("Datetime",axis=1)
df['']=dates
df=df.set_index('')
df

,Close,High,Low,Open,Volume
,,,,,
2024-12-16 15:00:00,132.519897,134.399994,130.449997,134.164993,49916873
2024-12-16 15:30:00,132.290100,132.859100,130.419998,132.509995,23941819
2024-12-16 16:00:00,131.550003,132.460007,131.330093,132.300003,14103584
2024-12-16 16:30:00,130.929901,132.169998,130.889999,131.550507,13065794
2024-12-16 17:00:00,131.380096,131.429993,130.690094,130.920105,11423976
...,...,...,...,...,...
2025-01-13 19:00:00,132.139999,132.630005,132.009995,132.520004,5925261
2025-01-13 19:30:00,132.294998,132.397003,131.570007,132.130203,7234556
2025-01-13 20:00:00,132.461197,132.529999,131.940002,132.299500,6923899


In [5]:
df=df.reindex(columns=["Open","High","Low","Close","Volume"])
print(df.head())

                           Open        High         Low       Close    Volume
                                                                             
2024-12-16 15:00:00  134.164993  134.399994  130.449997  132.519897  49916873
2024-12-16 15:30:00  132.509995  132.859100  130.419998  132.290100  23941819
2024-12-16 16:00:00  132.300003  132.460007  131.330093  131.550003  14103584
2024-12-16 16:30:00  131.550507  132.169998  130.889999  130.929901  13065794
2024-12-16 17:00:00  130.920105  131.429993  130.690094  131.380096  11423976


In [27]:
import plotly.graph_objects as go

def plot_backtest_results(df, trades):
    fig = go.Figure(data=[go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Candlesticks'
    )])

    for index, trade in trades.iterrows():
        if trade['Size'] > 0:
            fig.add_trace(go.Scatter(
                x=[trade['EntryTime']],
                y=[trade['EntryPrice']],
                mode='markers',
                marker=dict(color='green', size=10),
                name='Buy'
            ))
        elif trade['Size'] < 0:
            fig.add_trace(go.Scatter(
                x=[trade['ExitTime']],
                y=[trade['ExitPrice']],
                mode='markers',
                marker=dict(color='red', size=10),
                name='Sell'
            ))

    fig.update_layout(
        title='Backtest Results',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False
    )

    fig.show()


In [28]:
class EmaStrat(Strategy):
    fast_Ema = 4
    mid_Ema = 9
    slow_Ema = 18
    avg_Vol = 5
    take_profit = 0.05
    stop_loss = 0.02

    def init(self):
        self.fast_ema = self.I(ta.ema, pd.Series(self.data.Close), self.fast_Ema)
        self.mid_ema = self.I(ta.ema, pd.Series(self.data.Close), self.mid_Ema)
        self.slow_ema = self.I(ta.ema, pd.Series(self.data.Close), self.slow_Ema)
        self.avg_vol = self.I(ta.sma, pd.Series(self.data.Volume), self.avg_Vol)

    def next(self):
        if self.position:
            if self.position.pl_pct >= self.take_profit or self.position.pl_pct <= -self.stop_loss:
                self.position.close()
                print(f"Closed position at {self.data.Close[-1]} with P/L: {self.position.pl_pct:.2%}")

            elif self.fast_ema[-1] <= self.mid_ema[-1] or self.mid_ema[-1] <= self.slow_ema[-1]:
                self.position.close()
                print(f"Closed position at {self.data.Close[-1]} due to EMA crossover")

        else:
            if self.fast_ema[-1] > self.mid_ema[-1] > self.slow_ema[-1] and self.avg_vol[-1] > self.data.Volume[-1]:
                self.buy()
                print(f"Opened position at {self.data.Close[-1]}")



In [29]:
bt = Backtest(df, EmaStrat,cash=1_000_000)
stats = bt.run()
print(stats)

Opened position at 135.56370544433594
Closed position at 132.6999969482422 with P/L: -2.11%
Opened position at 133.91000366210938
Closed position at 141.64190673828125 with P/L: 5.77%
Opened position at 140.82009887695312
Closed position at 139.02000427246094 due to EMA crossover
Opened position at 139.8800048828125
Closed position at 136.48500061035156 with P/L: -2.42%
Opened position at 139.02000427246094
Closed position at 137.41000366210938 due to EMA crossover
Opened position at 137.92129516601562
Closed position at 136.11000061035156 due to EMA crossover
Opened position at 137.3022003173828
Closed position at 144.62759399414062 with P/L: 5.34%
Opened position at 144.49530029296875
Closed position at 151.8887939453125 with P/L: 5.11%
Opened position at 151.09500122070312
Closed position at 149.51010131835938 due to EMA crossover
Start                     2024-12-16 15:00:00
End                       2025-01-13 21:00:00
Duration                     28 days 06:00:00
Exposure Time [%

In [33]:
trades = stats['_trades']
print(trades)
plot_backtest_results(df, trades)
bt.plot()


   Size  EntryBar  ExitBar  EntryPrice   ExitPrice           PnL  ReturnPct  \
0  7376        28       37  135.560104  132.710007 -21022.320312  -0.021025   
1  7310        56       79  133.910004  141.660004  56652.500000   0.057875   
2  7353        80       86  140.830002  139.009995 -13382.513855  -0.012923   
3  7308        92       99  139.875000  136.479996 -24810.691223  -0.024272   
4  7174       117      124  139.029999  137.600006 -10258.767456  -0.010285   
5  7157       141      146  137.919998  136.100006 -13025.683212  -0.013196   
6  7095       147      157  137.289993  144.619995  52006.362991   0.053391   
7  7101       158      167  144.500000  151.880005  52405.414673   0.051073   
8  7138       168      176  151.100006  152.199997   7851.734650   0.007280   

            EntryTime            ExitTime   Tag        Duration  
0 2024-12-18 16:00:00 2024-12-18 20:30:00  None 0 days 04:30:00  
1 2024-12-20 17:00:00 2024-12-24 15:30:00  None 3 days 22:30:00  
2 2024-12-2

GridPlot(id='p1396', ...)

In [40]:
returns = []
minutes_in_day = 24 * 60

for x in range(5 * minutes_in_day, len(df) + 1, minutes_in_day):
    data_slice = df.iloc[x - 5 * minutes_in_day : x]
    bt = Backtest(data_slice, EmaStrat, cash=1_000_000)
    stats = bt.run(fast_Ema=4, mid_Ema=9, slow_Ema=18, avg_Vol=5)
    trades = stats['_trades']
    returns.append(trades["ReturnPct"])
    print(f"Return for slice {x}: {trades['ReturnPct']}")
    print(f"Number of trades: {trades['Size']}")

returns_df = pd.DataFrame(returns, columns=["Return pct"])
print(returns_df)

Empty DataFrame
Columns: [Return pct]
Index: []


In [38]:
fig = px.box(returns_df, y="Return pct", points="all")
fig.update_layout(
    xaxis_title="Strategy",
    yaxis_title="Returns",
    title="Distribution of Returns from EMA Strategy"
)
fig.show()